# Conformer generator

In [ ]:
import veloxchem as vlx

We define a structure by using a SMILES string.

In [ ]:
molecule = vlx.Molecule.read_smiles(
    "CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C"
)  # this is SMILES for Penicillin G
molecule.show(atom_indices=True)

ConformerGenerator class can generate all possible conformations and apply energy minimization process with MMforcefield

In [ ]:
conf = vlx.ConformerGenerator()
conformers_dict = conf.generate(molecule)

show the lowest energy conformer

In [ ]:
conf.show_global_minimum()

 show more conformers

In [ ]:
conf.show_conformers(number=3, atom_indices=True)

we can provide resp charges as partial charges to improve the used MMforcefield quality

In [ ]:
basis = vlx.MolecularBasis.read(molecule, "6-31g*")
resp = vlx.RespChargesDriver()
resp.ostream.mute()
partial_charges = resp.compute(molecule, basis, 'resp')

In [ ]:
conf = vlx.ConformerGenerator()
conf.partial_charges = partial_charges
conformers_dict = conf.generate(molecule)
conf.show_global_minimum(atom_indices=True)

we can apply implicit solvent model for the energy minimization 

In [ ]:
conf = vlx.ConformerGenerator()
conf.ostream.mute()
conf.show_available_implicit_solvent_models()
conf.implicit_solvent_model = "hct"  #here we set Hawkins-Cramer-Truhlar GBSA model
conformers_dict = conf.generate(molecule)
conf.show_global_minimum(atom_indices=True)

show more conformers

In [ ]:
conf.show_conformers(number=3, atom_indices=True)

# Extract conformer from an MD simulation

Using the OpenMMDynamics class, we can perform Molecular Dynamics

In [ ]:
molecule = vlx.Molecule.read_xyz_file("data/phenylalanine.xyz")
molecule.show(atom_indices=True)

Once a force field is derived, different functions can be used which use the OpenMM library to perform molecular dynamics simulations. This is available in the OpenMMDynamics class. For instance to perform a molecular dynamics:*

In [ ]:
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule)
opm_dyn = vlx.OpenMMDynamics()
opm_dyn.create_system_from_molecule(molecule,
                                    ff_gen,
                                    filename='phenylalanine',
                                    residue_name='MOL')
opm_dyn.run_md(ensemble='NVE', temperature=300, timestep=2.0, nsteps=100000, traj_file='phenylalanine_md.pdb')

Moreover, we have developed a function to extract conformers from the MD simulation at high temperature. The user can define a number of snapshots to be saved and decide to optimize them or not. In addition, those snapshots can be filtered to avoid that the same conformer is present multiple timess by using the option unique conformers.

In [ ]:
opm_energies, opm_geometries = opm_dyn.conformational_sampling(ensemble='NVT', 
                                temperature=1000, 
                                timestep=2.0, 
                                nsteps=100000, 
                                snapshots=500,
                                unique_conformers=True,
                                qm_driver=None,
                                basis=None,
                                constraints=None)